## Если библиотеки уже установлены, то пропустите следующую клетку

In [ ]:
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install lxml
!{sys.executable} -m pip install pymorphy2

In [80]:
from urllib import request
import lxml.html
import pandas as pd
import datetime
import pymorphy2


In [81]:
url = "https://www.gismeteo.ru/weather-vladivostok-4877/10-days/"
output = request.urlopen(url).read().decode("utf-8") 
html = lxml.html.fromstring(output)

In [82]:
# заменяет тире на минус в числах и запятую на точку в датах для удобного парсинга
def r(text):
    return text.replace('−', '-').replace(',','.')

In [83]:
temperatures = html.find_class('value')
precipitations = html.find_class('w_precipitation__value')
winds = html.find_class('w_wind')
dates = html.find_class('w_date')
data = {}
indexes = ["минимальная температура","максимальная температура","скорость ветра","уровень осадков"]
m = datetime.date.today()
morph = pymorphy2.MorphAnalyzer()

In [84]:
for i in range(10):
    col = dates[i][0][1].text.strip().split(' ')[0]+"."+m.strftime('%m')+" "+dates[i][0][0].text
    temp_min = int(r(temperatures[i][0].text))
    temp_max = int(r(temperatures[i][1].text))
    wind = int(str(lxml.html.tostring(winds[i][0][0])).split('>')[-1].split('\\n')[1].strip())
    precipitation = float(r(precipitations[i].text.strip()))
    data[col] = [temp_min,temp_max,wind,precipitation]

In [85]:
df = pd.DataFrame(data,index=indexes)

In [86]:
df

,10.02 Сб,11.02 Вс,12.02 Пн,13.02 Вт,14.02 Ср,15.02 Чт,16.02 Пт,17.02 Сб,18.02 Вс,19.02 Пн
минимальная температура,-4.0,-7.0,-7.0,-4.0,-3.0,-4.0,-3.0,-5.0,-7.0,-7.0
максимальная температура,-8.0,-12.0,-13.0,-13.0,-11.0,-10.0,-8.0,-9.0,-11.0,-13.0
скорость ветра,15.0,17.0,15.0,15.0,12.0,13.0,12.0,12.0,16.0,17.0
уровень осадков,0.0,0.1,0.0,0.0,1.3,0.0,0.0,1.0,1.2,0.0


В день1 похолодает / потеплеет на X градус (-а, -ов) по сравнению с день2

В четверг в НазваниеГорода потеплеет на 7 градусов по сравнению со средой

Скорость ветра изменится на X единиц в день1 по сравнению с день2.

Скорость ветра изменится на 3 единицы в понедельник по сравнению с пятницей

Уровень осадков повысится / понизится на X единиц за Y дней.

Уровень осадков понится на 3.85 единиц за 7 дней



In [87]:
days = {'Пн' : 'понедельник', 
       'Вт' : 'вторник',
        'Ср' : 'среда', 
        'Чт' : 'четверг', 
        'Пт' : 'пятница', 
        'Сб' : 'суббота', 
        'Вс' : 'воскресенье'
       }

In [88]:
#В день1 похолодает / потеплеет на X градус (-а, -ов) по сравнению с день2
def temperature_template(day1_date, day2_date):
    #TODO day1 должно быть больше чем day2
    morph = pymorphy2.MorphAnalyzer()
    day1_name = days[day1_date[-2:]]
    day1_name = morph.parse(day1_name)[0]
    day1_name = day1_name.inflect({'accs'}).word
    
    day2_name = days[day2_date[-2:]]
    day2_name = morph.parse(day2_name)[0]
    day2_name = day2_name.inflect({'ablt'}).word
    
    temp1 = (df[day1_date][0] + df[day1_date][1]) // 2
    temp2 = (df[day2_date][0] + df[day2_date][1]) // 2
    diff = abs(temp1 - temp2)
    
    degree = 'градус'
    degree = morph.parse(degree)[0]
    degree = degree.make_agree_with_number(diff).word
    
    if (temp1 > temp2):
        verb = 'потеплеет'
    else:
        verb = 'похолодает'
        
    city = 'Владивосток'
    city = morph.parse(city)[0]
    city = city.inflect({'loct'}).word
    
    s = ' '.join(['В', day1_name, 'во', city.title(), verb, 'на', str(int(diff)), degree, 'по сравнению с', day2_name])
        
    return s
    

In [89]:
print(temperature_template('11.02 Вс', '10.02 Сб'))

В воскресенье во Владивостоке похолодает на 4 градуса по сравнению с субботой


In [90]:
# Скорость ветра изменится на X единиц в день1 по сравнению с день2
def wind_template(day1_date, day2_date):
    morph = pymorphy2.MorphAnalyzer()
    day1_name = days[day1_date[-2:]]
    day1_name = morph.parse(day1_name)[0]
    day1_name = day1_name.inflect({'accs'}).word
    
    day2_name = days[day2_date[-2:]]
    day2_name = morph.parse(day2_name)[0]
    day2_name = day2_name.inflect({'ablt'}).word
    
    wind1 = df[day1_date][2]
    wind2 = df[day2_date][2]
    diff = abs(wind1 - wind2)
    
    unit = 'единица'
    unit = morph.parse(unit)[0]
    unit = unit.make_agree_with_number(diff).word
    
    s = ' '.join(['Скорость ветра изменится на', str(int(diff)), unit, 'в', day1_name, 'по сравнению с', day2_name])
        
    return s
    

In [91]:
print(wind_template('11.02 Вс', '10.02 Сб'))

Скорость ветра изменится на 2 единицы в воскресенье по сравнению с субботой


In [94]:
# Уровень осадков повысится / понизится на X единиц за Y дней.
from datetime import datetime

def precipitation_template(day1_date, day2_date):
    #TODO day1 должно быть больше чем day2
    morph = pymorphy2.MorphAnalyzer()

    date_format = "%d.%m"
    day1 = datetime.strptime(day1_date[:5], date_format)
    day2 = datetime.strptime(day2_date[:5], date_format)
    delta = int((day1 - day2).days)

    prec1 = df[day1_date][3]
    prec2 = df[day2_date][3]
    diff = abs(prec1 - prec2)
    
    unit = 'единица'
    unit = morph.parse(unit)[0]
    unit = unit.make_agree_with_number(diff).word
    
    days_count = 'дни'
    days_count = morph.parse(days_count)[0]
    days_count = days_count.make_agree_with_number(delta).word
    
    if (prec1 > prec2):
        verb = 'повысится'
    else:
        verb = 'понизится'
    
    s = ' '.join(['Уровень осадков', verb, 'на', str(diff), unit, 'за', str(delta), days_count])
        
    return s

In [98]:
print(precipitation_template('14.02 Ср', '10.02 Сб'))

Уровень осадков повысится на 1.3 единиц за 4 дня


In [96]:
butyavka = morph.parse('Владивостоке')[0]
print(butyavka)

Parse(word='владивостоке', tag=OpencorporaTag('NOUN,inan,masc,Geox sing,loct'), normal_form='владивосток', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'владивостоке', 73, 5),))


In [62]:
from datetime import datetime
date_format = "%d.%m"
a = datetime.strptime('10.02', date_format)
b = datetime.strptime('12.02', date_format)
delta = b - a
print(int(delta.days)) # that's it

2


Генераторы текстов широко используются при разработке и поисковой оптимизации сайтов: для генерации названий, описаний, и содержимого целых сайтов с помощью доргенов (генераторов дорвеев).

Существуют крупные англоязычные сайты, зарабатывающие на размещении рекламы, на которых весь контент пишут не журналисты, а боты — статьи автоматическим рерайтятся из других источников. Примеры таких сайтов: Demand Mediaruen и Associated Contentruen[3]. Русский язык, в отличие от английского, имеет сложную морфологию, поэтому появление подобных ботов-рерайтеров в рунете сильно осложнено.

Виртуальные собеседники (чат-боты) — программы, предназначенные имитировать общение в чатах. Они массово применяются для рассылки спама в соцсетях (спам-боты), а также как автоответчики, способные реагировать на множества ключевых слов по разным сценариям.


